<a href="https://colab.research.google.com/github/Matonice/30-Days-of-Transformer/blob/main/Semantic_Search_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install faiss-gpu
!pip install gradio

In [ ]:
from datasets import load_dataset
from huggingface_hub import hf_hub_url
import gradio as gr
from transformers import AutoTokenizer, AutoModel

In [ ]:
#downloading the dataset from the hubb
data_files = hf_hub_url(
    repo_id="lewtun/github-issues",
    filename="datasets-issues-with-comments.jsonl",
    repo_type="dataset",
)

In [ ]:
#Loading in the dataset
issues_dataset = load_dataset("json", data_files=data_files, split="train")
issues_dataset

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})

In [ ]:
#Since the github api return both pull requests and issues as issues, we need to filter out the pull request
issues_dataset = issues_dataset.filter(
    lambda x: (x["is_pull_request"] == False and len(x["comments"]) > 0)
)
issues_dataset

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 808
})

In [ ]:
#removing some columns and leaving the ones that are important to us
columns = issues_dataset.column_names
columns_to_keep = ["title", "body", "html_url", "comments"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)

issues_dataset = issues_dataset.remove_columns(columns_to_remove)
issues_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 808
})

In [ ]:
#Our comments column contains list of comments, we need to explode it so that each row as a single comment

issues_dataset.set_format("pandas")
df = issues_dataset[:]

comments_df = df.explode("comments", ignore_index=True)
comments_df.head()

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues...,Protect master branch,"Cool, I think we can do both :)",After accidental merge commit (91c55355b634d0d...
1,https://github.com/huggingface/datasets/issues...,Protect master branch,@lhoestq now the 2 are implemented.\r\n\r\nPle...,After accidental merge commit (91c55355b634d0d...
2,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Hi ! I guess the caching mechanism should have...,## Describe the bug\r\nAfter upgrading to data...
3,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,"If it's easy enough to implement, then yes ple...",## Describe the bug\r\nAfter upgrading to data...
4,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Well it can cause issue with anyone that updat...,## Describe the bug\r\nAfter upgrading to data...


In [ ]:
#Switching back to dataset format
from datasets import Dataset

comments_dataset = Dataset.from_pandas(comments_df)
comments_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 2964
})

In [ ]:
#creating another column for the length of each comments

comments_dataset = comments_dataset.map(
    lambda x: {"length_comment": len(x["comments"].split())}
)

  0%|          | 0/2964 [00:00<?, ?ex/s]

In [ ]:
#removing columns with comment lenght less than 15

comments_dataset = comments_dataset.filter(
    lambda x: x["length_comment"] > 15
)
comments_dataset

  0%|          | 0/3 [00:00<?, ?ba/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'length_comment'],
    num_rows: 2175
})

In [ ]:
def concatenate_text(examples):
    return {
        "text": examples["title"]
        + " \n "
        + examples["body"]
        + " \n "
        + examples["comments"]
    }


comments_dataset = comments_dataset.map(concatenate_text)
comments_dataset

  0%|          | 0/2175 [00:00<?, ?ex/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'length_comment', 'text'],
    num_rows: 2175
})

**Creating Text embeddings**

In [ ]:
model_checkpoint = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModel.from_pretrained(model_checkpoint)

In [ ]:
#connecting to  gpu

import torch

device = torch.device("cuda")
model = model.to(device)

In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [ ]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
#converting the comments into embeddings
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

  0%|          | 0/2175 [00:00<?, ?ex/s]

In [ ]:
#creating a faiss index
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'length_comment', 'text', 'embeddings'],
    num_rows: 2175
})

In [ ]:
question = "How can i load a dataset online"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [ ]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [ ]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [ ]:
samples_df

,html_url,title,comments,body,length_comment,text,embeddings,scores
4,https://github.com/huggingface/datasets/issues...,Discussion using datasets in offline mode,Requiring online connection is a deal breaker ...,"`datasets.load_dataset(""csv"", ...)` breaks if ...",57,Discussion using datasets in offline mode \n `...,"[-0.47318071126937866, 0.24578367173671722, -0...",32.490238
3,https://github.com/huggingface/datasets/issues...,Discussion using datasets in offline mode,I opened a PR that allows to reload modules th...,"`datasets.load_dataset(""csv"", ...)` breaks if ...",179,Discussion using datasets in offline mode \n `...,"[-0.47164806723594666, 0.2902272641658783, -0....",31.464340
2,https://github.com/huggingface/datasets/issues...,Discussion using datasets in offline mode,"The local dataset builders (csv, text , json a...","`datasets.load_dataset(""csv"", ...)` breaks if ...",38,Discussion using datasets in offline mode \n `...,"[-0.44908538460731506, 0.20950642228126526, -0...",30.439634
1,https://github.com/huggingface/datasets/issues...,Discussion using datasets in offline mode,"> here is my way to load a dataset offline, bu...","`datasets.load_dataset(""csv"", ...)` breaks if ...",76,Discussion using datasets in offline mode \n `...,"[-0.499260276556015, 0.22699761390686035, -0.0...",29.233568
0,https://github.com/huggingface/datasets/issues...,Discussion using datasets in offline mode,"here is my way to load a dataset offline, but ...","`datasets.load_dataset(""csv"", ...)` breaks if ...",47,Discussion using datasets in offline mode \n `...,"[-0.49025776982307434, 0.2288963347673416, -0....",28.624367


In [ ]:
len(samples)

7

In [ ]:
string = ""
for _, row in samples_df.iterrows():
    string += f"COMMENT: {row.comments}"
    string += f"SCORE: {row.scores}"
    string += f"TITLE: {row.title}"
    string += f"URL: {row.html_url}"
    string += "=" * 50
    string += "\n"
print(string)

COMMENT: Requiring online connection is a deal breaker in some cases unfortunately so it'd be great if offline mode is added similar to how `transformers` loads models offline fine.

@mandubian's second bullet point suggests that there's a workaround allowing you to use your offline (custom?) dataset with `datasets`. Could you please elaborate on how that should look like?SCORE: 32.490238189697266TITLE: Discussion using datasets in offline modeURL: https://github.com/huggingface/datasets/issues/824==================================================
COMMENT: I opened a PR that allows to reload modules that have already been loaded once even if there's no internet.

Let me know if you know other ways that can make the offline mode experience better. I'd be happy to add them :) 

I already note the "freeze" modules option, to prevent local modules updates. It would be a cool feature.

----------

> @mandubian's second bullet point suggests that there's a workaround allowing you to use your

In [ ]:
def search(question):
  question_embedding = get_embeddings([question]).cpu().detach().numpy()

  scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
  )

  samples_df = pd.DataFrame.from_dict(samples)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)

  string = ""
  for _, row in samples_df.iterrows():
    string += f"COMMENT: {row.comments}"
    string += f"SCORE: {row.scores}"
    string += f"TITLE: {row.title}"
    string += f"URL: {row.html_url}"
    string += "=" * 50
    string += "\n"

  return string

In [ ]:
demo = gr.Interface(search, inputs=gr.inputs.Textbox(),
                    outputs = gr.outputs.Textbox(),
                    title='Datasets issues search engine')

/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import 

In [ ]:
if __name__ == '__main__':
  demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://20580.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces
